In [1]:
import numpy as np
import metropolishastings as mh
from state import State

We use the Metropolis-Hastings algorithm to draw from a discrete random variable $X$ that takes 4 values, with the following pmf:

- $P(X=1) = 20 / K$,
- $P(X=2) = 8 / K$,
- $P(X=3) = 3 / K$,
- $P(X=4) = 1 / K$,

where $K$ is the normalization factor that is such that the probabilities sum up to one. In this simple example, the value of $K$ can be calculated ($K=32$). But the point is that the MH algorithm does not need it. It works with unnormalized probabilities.

The generating Markov process is a random walk jumping from one state to the next with equal probability.

We start four independent Markov processes, each staring from a different state.

In [3]:
initialStates = [
    simpleState(1), 
    simpleState(2), 
    simpleState(3), 
    simpleState(4)
]
numberOfDraws = 100000
maxNumberOfIterations = 10

We apply the MH algorithm.

In [4]:
draws, estimates, convergence, numberOfTrials = mh.MetropolisHastings(
    initialStates,
    numberOfDraws,
    maxNumberOfIterations,
)

Warmup


100%|██████████| 100000/100000 [00:01<00:00, 70358.91it/s]


Trial 0 with 100000 draws
Generated draws: (4, 100000, 4)
Potential scale reduction: [1.00004676 1.00000681 1.00003289 1.00000657]
    should be at most 1.1
Effective number of simulation draws: [ 97109.24472324 124025.5450491  177377.27984322 233238.93063331]
    should be at least 40


Here are the estimates for the frequency of each state.

In [5]:
estimates

array([0.625395 , 0.2492025, 0.0939275, 0.031475 ])

We calculate the theoretical probabilities for comparison.

In [7]:
weights = [20, 8, 3, 1]
prob = np.array(weights)
prob = prob / prob.sum()
prob

array([0.625  , 0.25   , 0.09375, 0.03125])

With this simple example, the MArkov porcess had apparently reached stationarity with the first set of draws that was generated. Apparently, because the criteria used are heuristics, and not exact. 

In [8]:
numberOfTrials

1

Here are the draws

In [9]:
draws

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
draws.shape

(4, 400000)

We can verify the the mean of each series corresponds to the estimates reported above

In [11]:
draws[0].mean()

0.625395

In [12]:
draws[1].mean()

0.2492025

In [13]:
draws[2].mean()

0.0939275

In [14]:
draws[3].mean()

0.031475